Introduction

## 1.  Extract info and prepare dataframe


In [1]:
!pip install bs4

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Parse the html data:

In [4]:
soup = BeautifulSoup(html_data, "html.parser")
#print(soup.prettify())

Extract data from the table into three relevant columns:

In [5]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not Assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = ((str((row.span.text).split('(')[1:]).replace(' /',',')).replace(')',' ')).strip(' ')

#        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
#print(table_contents)

Save the data to a dataframe:

In [6]:
df=pd.DataFrame(table_contents)

In [7]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(14)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,[]
1,M2A,Not assigned,[]
2,M3A,North York,['Parkwoods ']
3,M4A,North York,['Victoria Village ']
4,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']"
5,M6A,North York,"['Lawrence Manor, Lawrence Heights ']"
6,M7A,Queen's Park,['Ontario Provincial Government ']
7,M8A,Not assigned,[]
8,M9A,Etobicoke,['Islington Avenue ']
9,M1B,Scarborough,"['Malvern, Rouge ']"


(Still need to remove brackets and quote marks]

Remove rows where borough not assigned:

In [8]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace=True)
#  reset index
df = df.reset_index(drop=True)
df.shape

(103, 3)

In [9]:
#!pip install geocoder
# revisit geocoder

In [10]:
#  Obtain coordinates for Toronto neighborhoods

#postal_data = pd.read_csv('http://cocl.us/Geospatial_data') 
#postal_data.head()

# import csv from local machine
toronto_coords = pd.read_csv("C:\\Users\\steph\\pythonstuff\\Geospatial_Coordinates.csv")
toronto_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [11]:
df = pd.merge(df, toronto_coords, on='Postal Code')

Join the dataframe of neighborhoods with the latitude and longitude coordinates:

In [12]:
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,['Parkwoods '],43.753259,-79.329656
1,M4A,North York,['Victoria Village '],43.725882,-79.315572
2,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']",43.654260,-79.360636
3,M6A,North York,"['Lawrence Manor, Lawrence Heights ']",43.718518,-79.464763
4,M7A,Queen's Park,['Ontario Provincial Government '],43.662301,-79.389494
5,M9A,Etobicoke,['Islington Avenue '],43.667856,-79.532242
6,M1B,Scarborough,"['Malvern, Rouge ']",43.806686,-79.194353
7,M3B,North York,['Don Mills North'],43.745906,-79.352188
8,M4B,East York,"['Parkview Hill, Woodbine Gardens ']",43.706397,-79.309937
9,M5B,Downtown Toronto,"['Garden District, Ryerson ']",43.657162,-79.378937


In [13]:
df.shape    # 103 x 5;  now 103x9

(103, 5)

Before exploring the data, let's download the dependencies that we will need.

In [14]:
#import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#! pip install folium==0.5.0
#import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Let's create a map of Toronto.  First, get the coordinates for the main boroughs of Toronto:

In [15]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="course_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto, Ontario, Canada are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto, Ontario, Canada are 43.6534817, -79.3839347.


Let's segment and cluster certain neighborhoods in Toronto. We slice the original dataframe and create a new dataframe of Toronto neighborhooddata.

In [16]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Use Foursquare Foursquare API to explore the neighborhoods and segment them.

In [17]:
CLIENT_ID = '3VH3RBUX11UHLC4BYM2THX4L52OB2U2D2QX1YKGMXQ0ZIPUR' # your Foursquare ID
CLIENT_SECRET = 'GPU4351JMFH2ZRBFW4S04CN3TYB2HLINMZDHX0T11IEWXID1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3VH3RBUX11UHLC4BYM2THX4L52OB2U2D2QX1YKGMXQ0ZIPUR
CLIENT_SECRET:GPU4351JMFH2ZRBFW4S04CN3TYB2HLINMZDHX0T11IEWXID1


Foursquare API:

https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT

Let's explore a neighborhood in our dataframe.

Get the neighborhood's name and coordintes.


In [18]:
df.loc[2, 'Neighborhood']

"['Regent Park, Harbourfront ']"

In [19]:
neighborhood_latitude = df.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Regent Park, Harbourfront '] are 43.6542599, -79.3606359.


Now, let's get the top 100 venues that are in ['Parkwoods'] within a radius of 500 meters.

In [20]:
# First, let's create the GET request URL. Name your URL url.
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c618ee90458e010bf41cfe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

<ipython-input-23-53c7dde730be>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,The Distillery Historic District,Historic Site,43.650244,-79.359323
6,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
7,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
8,Sumach Espresso,Coffee Shop,43.658135,-79.359515
9,Arvo,Coffee Shop,43.649963,-79.361442


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


## 2.  Explore Neighborhoods in Toronto.

Let's create a function to repeat the same process to all the neighborhoods in Toronto.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [26]:
# type your answer here
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


['Parkwoods ']
['Victoria Village ']
['Regent Park, Harbourfront ']
['Lawrence Manor, Lawrence Heights ']
['Ontario Provincial Government ']
['Islington Avenue ']
['Malvern, Rouge ']
['Don Mills North']
['Parkview Hill, Woodbine Gardens ']
['Garden District, Ryerson ']
['Glencairn ']
['West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale ']
['Rouge Hill, Port Union, Highland Creek ']
['Don Mills South', 'Flemingdon Park ']
['Woodbine Heights ']
['St. James Town ']
['Humewood-Cedarvale ']
['Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood ']
['Guildwood, Morningside, West Hill ']
['The Beaches ']
['Berczy Park ']
['Caledonia-Fairbanks ']
['Woburn ']
['Leaside ']
['Central Bay Street ']
['Christie ']
['Cedarbrae ']
['Hillcrest Village ']
['Bathurst Manor, Wilson Heights, Downsview North ']
['Thorncliffe Park ']
['Richmond, Adelaide, King ']
['Dufferin, Dovercourt Village ']
['Scarborough Village ']
['Fairview, Henry Farm, Oriole ']
['Northwood Park, York Un

Let's check how many venues were returned for each neighborhood

In [27]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"[""Clarks Corners, Tam O'Shanter, Sullivan ""]",9,9,9,9,9,9
"[""Milliken, Agincourt North, Steeles East, L'Amoreaux East ""]",3,3,3,3,3,3
"[""Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East ""]",1,1,1,1,1,1
"[""Steeles West, L'Amoreaux West ""]",11,11,11,11,11,11
['Agincourt '],5,5,5,5,5,5
"['Alderwood, Long Branch ']",7,7,7,7,7,7
"['Bathurst Manor, Wilson Heights, Downsview North ']",16,16,16,16,16,16
['Bayview Village '],4,4,4,4,4,4
"['Bedford Park, Lawrence Manor East ']",23,23,23,23,23,23


Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


## 3.  Analyze each neighborhood:

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.


In [30]:
toronto_onehot.shape  # 2012x 256

(1992, 263)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
# toronto_grouped        takes a while to render

In [32]:
toronto_grouped.shape    #  99 x 256

(100, 263)

Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----["Clarks Corners, Tam O'Shanter, Sullivan "]----
                  venue  freq
0           Pizza Place  0.22
1    Chinese Restaurant  0.11
2                  Bank  0.11
3  Fast Food Restaurant  0.11
4           Gas Station  0.11


----["Milliken, Agincourt North, Steeles East, L'Amoreaux East "]----
                 venue  freq
0         Intersection  0.33
1                 Park  0.33
2           Playground  0.33
3   Mexican Restaurant  0.00
4  Monument / Landmark  0.00


----["Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East "]----
                 venue  freq
0       Baseball Field   1.0
1          Yoga Studio   0.0
2  Moroccan Restaurant   0.0
3  Martial Arts School   0.0
4       Massage Studio   0.0


----["Steeles West, L'Amoreaux West "]----
                  venue  freq
0  Fast Food Restaurant  0.18
1           Coffee Shop  0.09
2        Breakfast Spot  0.09
3     Electronics Store  0.09
4 

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.


In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"[""Clarks Corners, Tam O'Shanter, Sullivan ""]",Pizza Place,Chinese Restaurant,Gas Station,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Bank,Italian Restaurant,Dessert Shop,Dim Sum Restaurant
1,"[""Milliken, Agincourt North, Steeles East, L'A...",Park,Playground,Intersection,Electronics Store,Eastern European Restaurant,Drugstore,Escape Room,Donut Shop,Deli / Bodega,Dog Run
2,"[""Old Mill South, King's Mill Park, Sunnylea, ...",Baseball Field,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
3,"[""Steeles West, L'Amoreaux West ""]",Fast Food Restaurant,Electronics Store,Coffee Shop,Bank,Pharmacy,Pizza Place,Breakfast Spot,Supermarket,Chinese Restaurant,Sandwich Place
4,['Agincourt '],Lounge,Breakfast Spot,Skating Rink,Clothing Store,Latin American Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.


In [36]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto data [df] to add lat/long for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# remove NAs and cast to type int
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].fillna(0).astype('int') 

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,['Parkwoods '],43.753259,-79.329656,2,Park,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
1,M4A,North York,['Victoria Village '],43.725882,-79.315572,0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Pub,Greek Restaurant,Sandwich Place,Bank,Spa,Beer Store
3,M6A,North York,"['Lawrence Manor, Lawrence Heights ']",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Coffee Shop,Boutique,Distribution Center,Dessert Shop,Dim Sum Restaurant
4,M7A,Queen's Park,['Ontario Provincial Government '],43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Theater,Sandwich Place,Salad Place,Diner,Burger Joint,Japanese Restaurant


Finally, let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


### Cluster 0: Coffee Shops, Restaurants and Retail

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Pub,Greek Restaurant,Sandwich Place,Bank,Spa,Beer Store
3,North York,0,Clothing Store,Furniture / Home Store,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Coffee Shop,Boutique,Distribution Center,Dessert Shop,Dim Sum Restaurant
4,Queen's Park,0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Theater,Sandwich Place,Salad Place,Diner,Burger Joint,Japanese Restaurant
5,Etobicoke,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Scarborough,0,Fast Food Restaurant,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
7,North York,0,Japanese Restaurant,Gym,Café,Caribbean Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
8,East York,0,Pizza Place,Gastropub,Pharmacy,Bank,Athletics & Sports,Flea Market,Café,Intersection,Pet Store,Gym / Fitness Center
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Sandwich Place,Café,Hotel,Pizza Place,Japanese Restaurant,Cosmetics Shop,Bank,Ramen Restaurant
10,North York,0,Park,Japanese Restaurant,Pizza Place,Bakery,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


###  Cluster 1: Shopping Mall 

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,1,Shopping Mall,Women's Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


### Cluster 2: Parks, Snacks and Distribution Centers

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
21,York,2,Park,Women's Store,Pool,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega
35,East York/East Toronto,2,Park,Intersection,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
40,North York,2,Park,Snack Place,Airport,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
52,North York,2,Park,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
61,Central Toronto,2,Park,Bus Line,Swim School,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
64,York,2,Park,Jewelry Store,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
66,North York,2,Park,Moving Target,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
77,Etobicoke,2,Park,Sandwich Place,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store,Deli / Bodega
91,Downtown Toronto,2,Park,Trail,Playground,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center


### Cluster 3: Baseball Fields, Department Stores, Eastern European Restaurants

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,3,Baseball Field,Food Truck,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store,Falafel Restaurant
57,North York,3,Baseball Field,Furniture / Home Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Falafel Restaurant,Doner Restaurant
101,Etobicoke,3,Baseball Field,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant


### Cluster 4:  Playgrounds, Dog Runs, Stores

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,4,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
85,Scarborough,4,Park,Playground,Intersection,Electronics Store,Eastern European Restaurant,Drugstore,Escape Room,Donut Shop,Deli / Bodega,Dog Run
